### 4. Topic Modeling met Gensim

Het doel van dit script is om het LDA Topic Modeling algoritme toe te passen op het corpus verzameld in het json-bestand. Hiervoor maken we gebruik van de Gensim Topic Modeling bibliotheek. Eerst moet echter het corpus worden opgeschoond. Deze stap, ook wel de preprocessing van het corpus genoemd, wordt gedaan gebruikmakende van spaCy, een natural language processing (NLP) python bibliotheek, en Gensim. Naast de python bibliotheek downloaden we ook een model van spaCy, kijk op  https://spacy.io/models/ welke modellen er beschikbaar zijn. LET OP: kies een talenmodel dat het beste past bij je corpus. De spaCy modellen zijn allemaal getraind op moderne teksten. Hieronder werken we standaard met het kleine Nederlandse taalmodel van spaCy (nl_core_news_sm). Ten slotte wordt ook een aantal manieren gepresenteerd om de resultaten van het LDA topic model te visualiseren en te beoordelen. Dit script is grotendeels gebaseerd op het artikel *Topic Modeling with Gensim* van S. Prabhakaran en de YouTube tutorials *Topic Modeling for DH* en de GitHub *Implementing LDA in Python* van W. Mattingly. 

Dit script behoeft de volgende installaties:

- `pip install gensim`
- `pip install spacy`
- `pip install pyldavis`
- `python -m spacy download nl_core_news_sm`

Documentatie:
- gensim https://radimrehurek.com/gensim/auto_examples/index.html#documentation
- spacy https://spacy.io/, https://spacy.io/usage/spacy-101
- pyldavis: https://pyldavis.readthedocs.io/en/latest/, https://pyldavis.readthedocs.io/en/latest/modules/API.html


Bronnen:
- S. Kapadia, *Topic Modeling in Python: Latent Dirichlet Allocation (LDA)* https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0 
- S. Kapadia, *Evaluate Topic Models: Latent Dirichlet Allocation (LDA)* https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
- W. Mattingly, *Implementing LDA in Python*. Github: https://github.com/wjbmattingly/topic_modeling_textbook/blob/main/03_03_lda_model_demo.ipynb, 
- W. Mattingly, *Topic Modeling for DH*. YouTube tutorials: https://www.youtube.com/watch?v=TKjjlp5_r7o&list=PL2VXyKi-KpYttggRATQVmgFcQst3z6OlX&index=10) 
- S. Prabhakaran, *Topic Modeling with Gensim* (Python)
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
- S. Prabhakaran, *Topic modeling visualization - How to present the results of LDA models?*
https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/



##### Stap 1: importeren van de benodigde python bibliotheken 

In [ ]:
#spacy voor natural language processing
import spacy

#Gensim voor topic modeling
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

#pyLDAvis voor visualisaties
import pyLDAvis
import pyLDAvis.gensim_models

#overige benodigdheden
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

##### Stap 2: inlezen van de data uit json (corpus)

In het onderstaande celblokje laden we de teksten uit het corpus, oftewel het json bestand. De teksten vormen de data waarmee het Topic Modeling algoritme na de corpusbewerking en -opschoning aan de slag gaat. 

In [ ]:
corpus_path = '' #VUL IN: plaats tussen de aanhalingstekens het pad naar het json-bestand met de versie van het corpus waar je de topics van wilt bepalen
              #dit kan het json-bestand zijn van het hele corpus dat je in Hst 3 gemaakt hebt, of het json-bestand van het geschoonde corpus dat je in Hst 4 gemaakt hebt

def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

data = load_data(corpus_path)["texts"] 
print(data[0][:100]) #ter controle krijg je de eerste 100 tekens uit het eerste document 

##### Stap 3: corpusbewerking en opschoning met spaCy en Gensim (preprocessing) 

In de onderstaande celblokjes wordt het corpus met behulp van spaCy en Gensim opgeschoond. Dé juiste corpusbewerking en opschoning bestaat niet. Voor elk corpus en topic model moet met de preprocessing geëxperimenteerd worden om uit te vogelen wat de beste keuzes zijn. Uiteindelijk is het doel van Topic Modeling om onderscheidende topics te generen, bestaande uit betekenisvolle woorden. De preprocessing of corpusopschoning draagt hier aan bij. 

Eerst laden we een Nederlands model en standaard stopwoordenlijst. Dit model is getraind op nieuwsberichten. Er zijn ook grotere modelen beschikbaar (nl_core_news_sm) die eventueel betere resultaten opleveren, zie https://spacy.io/models/nl voor meer informatie. De stopwoordenlijst kan worden uitgebreid met corpus-specifieke stopwoorden. Stel dat er bijvoorbeeld veel brieven of facturen met briefhoofden en adreseringen in je corpus zitten, worden deze vaak voorkomende woorden snel als topic gezien, maar zijn weinig betekenisvol. 

In [ ]:
nlp = spacy.load("nl_core_news_sm") 
nlp.Defaults.stop_words |= {"", "", ""} #VUL IN: tussen de aanhalingstekens kun je woorden plaatsen die je wilt toevoegen aan de stopwoordenlijst. Voeg er zoveel toe als je wilt telkens tussen aanhalingstekens en gescheiden door een komma 
print(nlp.Defaults.stop_words) #hier printen we de stopwoordenlijst om een indruk te krijgen van wat voor woorden spaCy als stopwoord aanmerkt. 

Daarna definiëren we de gewenste corpusopschoning/bewerking. spaCy deelt elk document in het corpus op in tokens. Deze tokens worden geannoteerd met de woordsoort of functie binnen de zin en op basis hiervan kunnnen bepaalde soort tokens verwijderd worden. Hier is gekozen voor de volgende opschoning:

- Behoud van alfabetische tokens, verwijderen van leestekens, interpunctie, en cijfers.
- Verwijderen van tokens geannoteerd als stopwoord. 
- Behoud van toegestane woordsoorten (allowed_postags): tokens aangeduid als zelfstandig naamwoorden, bijvoegelijk naamwoorden, werkwoorden en bijwoorden blijven behouden. Voor een groot corpus is het wellicht nodig om ook de werkwoorden te verwijderen uit het corpus. Zie de volgende websites voor meer informatie over woordsoortsherkenning (part of speech-tagging) en de verschillende woordsoortslabels binnen spaCy: https://spacy.io/usage/linguistic-features, https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/ 
- Lemmatiseren van overgebleven tokens: omzetten van woorden in woordenboekvorm. 

In [ ]:
# Corpusbewerking 
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):  #hier definiëren we de woordsoorten die we willen behouden. 
    texts_out = []                                                         
    for text in texts:                                                     
        nlp.max_length = len(text)  
        doc = nlp(text)
        new_text = []
        for token in doc: #hier deelt spaCy elk document op in tokens, die vervolgens worden geannoteerd
            if token.is_alpha: #behoud alfabetische tokens, haal deze regel weg als je ook cijfers wilt behouden
                if not token.is_stop: #behoud tokens die NIET in stopwoordenlijst staan
                    if token.pos_ in allowed_postags: #behoud tokens geduid met de hierboven toegestane woordsoorten
                        new_text.append(token.lemma_) #omzetten van woorden in woordenboekvorm 
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data) #hier passen we de bovenstaande corpusbewerking toe op het corpus
print(lemmatized_texts[0]) #print van eerste document 

De Topic Modeling bibliotheek Gensim biedt ook verschillende mogelijkheden om het corpus verder op te schonen. Met utils.simple_preprocess worden in het onderstaande celblokje de overgebleven tokens van elkaar gescheiden met komma's en apostroffen.

In [ ]:
# Verdere opschoning van het corpus
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print(data_words[0]) #eerste document uit corpus, de documenten hebben al een aardige transformatie ondergaan. 

##### Stap 4: Vaste verbindingen (bigrams en trigrams)

In het onderstaande celblokje berekenen we de vaste verbindingen (phrases) binnen het corpus met behulp van Gensim. Vaste verbindingen zijn woorden die vaak naast elkaar voorkomen. Hiermee wordt enige semantische betekenis of context behouden in de resultaten van het topic model. 
- Een bigram is een vaste verbinding bestaande uit twee woorden: (het) koninklijk huis
- Een trigram is een vaste verbinding bestaande uit drie woorden: Koninklijk(e) Luchtvaart Maatschappij 

Parameters:
- min_count: minimaal aantal keer dat woorden samen moeten voorkomen om als vaste verbinding gerekend te worden. Experimenteer met de min_count. 
- threshold: score threshold voor vaste verbindingen, hogere threshold betekent minder vaste verbindingen. Experimenteer met de juiste threshold

In [ ]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100) 

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print(data_bigrams_trigrams[0])

##### Stap 5: Bag of words representatie van het corpus

Om het algoritme het corpus te laten begrijpen, moeten de documenten op een andere manier gerepresenteerd worden. Voor het LDA algoritme moeten de documenten worden gepresenteerd als bag-of-words. Deze representatie wordt zo genoemd, omdat de volgorde van de woorden niet wordt meegerekend. In Gensim worden de documenten als bag-of-words gepresenteerd aan de hand van een dictionary (id2word) en een corpus. Het document wordt minder leesbaar voor de mens, maar beter te begrijpen voor het algoritme. De dictionary and het corpus worden in het onderstaande celblokje gecreëerd. 

- dictionary = elk woord in het gehele corpus krijgt een identificatie (id) nummer
- corpus = elk document gerepresenteerd aan de hand van het id nummer en de frequentie van de woorden binnen het document.

Tijdens deze stap is het ook mogelijk om de functie .filter_extremes binnen Gensim toe te passen. Met deze functie is het mogelijk om woorden uit de dictionary en het corpus te filteren die té uniek of té algemeen zijn. Zie https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.filter_extremes.html voor meer informatie over .filter_extremes

Parameters: 

- no_below: woorden filteren die in minder dan een X aantal documenten voorkomen
- no_above: filteren op basis van het percentage (0.XX) documenten waarin een woord voorkomt
- keep_n: behouden van de eerste X aantal van de meest frequente tokens (relevant bij hele grote corpora)




In [ ]:
id2word = corpora.Dictionary(data_bigrams_trigrams)
id2word.filter_extremes(no_below=5, no_above=0.5, keep_n=100000) 

corpus = []
for text in data_bigrams_trigrams:
    new = id2word.doc2bow(text)
    corpus.append(new)

print(corpus[0]) #eerste document gepresenteerd als bag-of-words aan de hand van id nummer en woordfrequenties 

In [ ]:
#vul bij de X een van de identificatie nummers uit het eerste document in om te zien welk woord het is 
word = id2word[[34][:1][0]]
print (word)

##### Stap 6: TF-IDF (term frequency - inverse document frequency)

TF-IDF is andere manier om woorden te behouden die uniek zijn voor een specifiek document en minder vaak voorkomen in het gehele corpus. Woorden krijgen een waarde op basis van de woordfrequentie binnen een document en de frequentie van datzelfde woord in het gehele corpus. Let op: TF-IDF is overbodig als filter_extremes is gebruikt, LDA topic modeling heeft ook al een soort TF-IDF functie in zich dus waarschijnlijk is TF-IDF niet nodig als stap. 


In [ ]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_words

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] 
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] 

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

##### Stap 7: Topic Modeling met Gensim

Nu is het eindelijk tijd om met Topic Modeling aan de slag te gaan. Daar is slechts een celblokje voor nodig. In dit blokje moet een aantal parameters gedefinieerd worden. Raadpleeg de haalbaarheidsstudie voor meer informatie over de paramaters of bezoek de onderstaande websites. 

- LDA model documententatie: https://radimrehurek.com/gensim/models/ldamodel.html 
- Ook mogelijk om LDA model te laten draaien met meerdere processor cores (kernen): https://radimrehurek.com/gensim/models/ldamulticore.html

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,   #corpus met documenten gerepresenteerd als bag-of-words
                                           id2word=id2word,  #dictionary: lijst met identificatie nummers van alle woorden
                                           num_topics=X,     #VUL IN: vervang X voor het aantal topics dat het algoritme zal identificeren 
                                           random_state=100, #seed voor het later kunnen reproduceren van de resultaten
                                           update_every=1,   #0 voor batch learning, > 1 voor online learning, zie: https://vitalflux.com/difference-between-online-batch-learning/
                                           iterations=100,   #aantal herhalingen 
                                           chunksize=len(corpus), #aantal gebruikte documenten per training 
                                           passes=100,       #aantal herhalingen gedurende een training 
                                           alpha="auto",     #LDA hyperparameter, zie haalbaarheidsstudie
                                           eta="symmetric")  #LDA hyperparameter, zie haalbaarheidsstudie



In [ ]:
#lda multicore model voor grotere corpora

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                        id2word=dictionary,
                                        num_topics=X,        #VUL IN: vervang X voor het aantal topics dat het algoritme zal identificeren
                                        iterations=100
                                        random_state=100,
                                        update_every=1
                                        chunksize=len(corpus),
                                        passes=100,
                                        alpha="auto",
                                        eta="symmetric",
                                        worker=X) #VUL IN: aantal workers is afhankelijk van de gebruikte computer, beste is om totaal aantal cores -1 te kiezen. 

##### Stap 8: Visualisatie van de resultaten

In de onderstaande code blokjes wordt een aantal manier gepresenteerd om de resultaten van het topic model te visualiseren:

- printen van de topics met pprint: topics met bijbehorende keywords
- printen van de topic distributie per document
- visualisatie van topic clusters en keywords met pyLDAvis

Een eerste stap om de resultaten van het topic model te bestuderen is door een lijst te printen van de topics met bijbehorende keywords. Hierbij kan gekeken worden of het topic model topics begrijpelijke, coherente en onderscheidende topics heeft gegenereerd. 

In [ ]:
#printen van topics met keywords
from pprint import pprint
pprint(lda_model.print_topics(num_topics = X, num_words = X)) #VUL IN: vervang de X-en voor het aantal topics en gewenste aantal keywords per topic 
doc_lda = lda_model[corpus]

Een tweede stap om de resultaten te bekijken is door de topic distributie over de documenten te bekijken. LDA Topic Modeling is ontworpen op basis van de gedachte dat een document uit meerdere topics bestaat. Met behulp van de onderstaande celblokjes wordt een csv-bestand gecreëerd waarin voor elk document wordt aangegeven in welke hoedanigheid de topics voorkomen in een document. Mocht je voor elk document alle topic percentages willen hebben, voeg dan minimum_probability=0.0 toe aan: 

``` python
topics = lda_model.get_document_topics(corpus, per_word_topics=True, minimum_probability=0.0)
```

Gebaseerd op de volgende GitHub: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/topic_methods.ipynb 

In [ ]:
#print de topic distributie per document 
topics = lda_model.get_document_topics(corpus, per_word_topics=True) #VUL IN: voeg hier minum_probability=0.0 toe, vergeet de , (komma) niet na per_word_topics=True
all_topics = [(doc_topics, word_topics, word_phis) for doc_topics, word_topics, word_phis in topics]

document_topics = []

for doc in all_topics:
    document_topics.append(doc[0])

print(document_topics)

In [ ]:
#bewaar de topic distributie per document in een csv-bestand

#lijst naar dataframe 
df = pd.DataFrame(document_topics)

#voeg documentnamen en eventueel de index en de tekst uit json toe aan de dataframe 
document_name = load_data(corpus_path)["doc_id"]
document_text = load_data(corpus_path)["texts"]

df.insert(0, "document_name", document_name) # voeg namen toe als eerste column (0)
df.insert(1, "document_text", document_text) # voeg tekst toe als tweede column (1)

#dataframe naar csv
dst_folder = '' + '/' #VUL IN: plaats tussen de eerste aanhalingstekens het pad naar de map waar je de csv wilt opslaan
file_name_csv = '' + '.csv' #VUL IN: plaats tussen de eerste aanhalingstekens de naam waaronder je de csv wilt opslaan
df.to_csv(dst_folder + file_name_csv)

Een derde stap is om de resultaten van het topic model te interpreteren is door middel van het bestuderen van een visualisatie. Met behulp van pyLDAvis worden de topics en keywords gepresenteerd als bollen of clusters. Voor meer informatie over pyLDAvis, zie:

- pyLDAvis documentatie: https://pyldavis.readthedocs.io/en/latest/, https://pyldavis.readthedocs.io/en/latest/modules/API.html
- Sievert, C. and Shirley, K. (2014): *LDAvis: A Method for Visualizing and Interpreting Topics*, ACL Workshop on Interactive Language Learning, Visualization, and Interfaces.
https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf
- Voor een meer toegankelijkere uitleg, google of zie: https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know

In [ ]:
#visualisatie met pyLDAvis
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim_models.prepare(lda_model,corpus,id2word)
vis

In [ ]:
#visualisatie opslaan als HTML
p = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(p, '' + '.html') #VUL IN: plaats tussen de eerste aanhalingstekens de bestandsnaam waaronder je het bestand wilt opslaan

Alternatief: met de onderstaande celblokjes wordt een csv-bestand gecreëerd waarin voor elk document wordt aangegeven wat het dominante topic is. Mocht je alleen het meest dominante topic per document willen zien, kan je deze manier gebruiken. Over het algemeen is het beter om te kunnen zien of een document uit meerdere topics bestaat, omdat het LDA model hier vanuit gaat. 

In [ ]:
# dominante topic voor elk document printen 
import re
import numpy as np
import pandas as pd
from pprint import pprint 

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    sent_topics_df = pd.DataFrame()

    # Pakt voor elk document het topic dat het meeste voorkomt
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Pakt het dominante topic, percentage dat het topic bijdraagt aan het document en de keywords 
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominante topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

file_name_csv2 = '' + '.csv' #VUL IN: plaats tussen de eerste aanhalingstekens de bestandsnaam waaronder je deze csv wilt opslaan

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_bigrams_trigrams)

df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(66)
df_dominant_topic.to_csv(dst_folder + file_name_csv2) 

##### Stap 9: Beoordeling resultaten met Coherence score

In het onderstaande celblokje wordt de coherence score voor het topic model berekend. De coherence score geeft een indicatie in hoeverre de topics begrijpbaar zijn voor een mens. Het model krijgt een coherence score op basis van de gelijkenis tussen de woorden binnen elk topic. Hoe hoger de coherence score, hoe uniformer de woorden binnen de topics zijn. Let op: de coherence score is slechts een indicator en kan de beoordeling van een mens niet perfect nabootsen. Beoordeel een model nooit enkel en alleen op basis van de coherence score. 

Zie de haalbaarheidsstudie voor meer informatie over de coherence score

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_bigrams_trigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

##### Stap 10: LDA model opslaan en laden

Als je eenmaal een topic model hebt waarmee je uit te voeten kan, is het mogelijk om deze op te slaan en op een later moment of in een ander script weer in te laden. 

Zie deze YouTube tutorial voor meer uitleg: https://www.youtube.com/watch?v=xADAr8pPQMI&list=PL2VXyKi-KpYttggRATQVmgFcQst3z6OlX&index=12 

In [ ]:
#opslaan van LDA topic model
file_name_model = '' + '.model' #VUL IN: plaats tussen de eerste aanhalingstekens de bestandsnaam waaronder je het model wilt opslaan
lda_model.save(dst_folder + file_name_model)

In [ ]:
#inladen van een LDA topic model
new_model = gensim.models.ldamodel.LdaModel.load(dst_folder + file_name_model)